1. pivot talbe of user movie table
2. calculate the weights between users
3. calculate the rating for i users

In [1]:
import pandas as pd
import numpy as np

In [2]:
rating = pd.read_csv('rating.csv', index_col=0)

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# link = pd.read_csv('link.csv', index_col=0)
# movie = pd.read_csv('movie.csv', index_col=0)
# genome_tags = pd.read_csv('genome_tags.csv', index_col=0)

In [4]:
rating.head()

,movieId,rating,timestamp
userId,,,
1,2,3.5,2005-04-02 23:53:47
1,29,3.5,2005-04-02 23:31:16
1,32,3.5,2005-04-02 23:33:39
1,47,3.5,2005-04-02 23:32:07
1,50,3.5,2005-04-02 23:29:40


In [5]:
flattened = pd.DataFrame(rating.to_records())

In [6]:
flattened.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
sample = flattened.head(100000)

In [8]:
user_movie_table = pd.pivot_table(sample, index=['userId'], columns=['movieId'])

In [9]:
user_movie_table.head()

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          NaN    3.5    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2          NaN    NaN    4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN    4.0   
5          NaN    3.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

         ...                                                                   \
movieId  ...   117511 117590 118354 118696 118900 118997 119141 125916 128488   
userId   ...                                                                    
1        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                
movieId 128594  
userId          
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  

[5 rows x 8227 columns]

In [10]:
#find two users pearson correlation

In [11]:
def weight_calculator(user_movie_table, index1, index2):
    """find pearson corr between two users
    Parameters
    --------------
    user_movie_table : pd.DataFrame
        user_movie rating info
    index1, index2 : Integer
        pointer in table
    """
    mu = user_movie_table.T
    user1 = mu[index1]
    user2 = mu[index2]
    common = mu[[index1,index2]].dropna()
    user1_watched = user1.dropna()
    user2_watched = user2.dropna()
    
    x1_mean = user1.mean()
    x2_mean = user2.mean()
    numerator = np.array(list(map(lambda x,y : (x-x1_mean)*(x-x2_mean), common[index1],common[index2]))).sum()
    
    left_corner = np.array(list(map(lambda x : (x-x1_mean)**2, user1_watched))).sum()**0.5
    right_corner = np.array(list(map(lambda x : (x-x2_mean)**2, user2_watched))).sum()**0.5
    denominator = left_corner * right_corner
    
    weight_between_two_users = numerator/denominator
    
    return weight_between_two_users

In [13]:
w = weight_calculator(user_movie_table, 1, 2)

In [14]:
w

0.014944306957911126

### => Note! I found the weight for a person to himself is not 1!

# Now, take the user1 as the target. Let's say I want to know whether I should recommend movie1 to him.

user3,6,8,10 has review for movie1

In [16]:
mu = user_movie_table.T
user1 = mu[1]

In [17]:
user1.head()

        movieId
rating  1          NaN
        2          3.5
        3          NaN
        4          NaN
        5          NaN
Name: 1, dtype: float64

In [18]:
user_movie_table.head(10)

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          NaN    3.5    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2          NaN    NaN    4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN    4.0   
5          NaN    3.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
6          5.0    NaN    3.0    NaN    NaN    NaN    5.0    NaN    NaN    NaN   
7          NaN    NaN    3.0    NaN    NaN    NaN    3.0    NaN    NaN    NaN   
8          4.0    NaN    5.0    NaN    NaN    3.0    NaN    NaN    NaN    4.0   
9          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
10         4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

         ...                                                                   \
movieId  ...   117511 117590 118354 118696 118900 118997 119141 125916 128488   
userId   ...                                                                    
1        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
6        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
7        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
8        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
9        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
10       ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                
movieId 128594  
userId          
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  
6          NaN  
7          NaN  
8          NaN  
9          NaN  
10         NaN  

[10 rows x 8227 columns]

In [20]:
w_1_3= weight_calculator(user_movie_table, 1, 3)
w_1_6= weight_calculator(user_movie_table, 1, 6)
w_1_8= weight_calculator(user_movie_table, 1, 8)
w_1_10= weight_calculator(user_movie_table, 1, 10)

In [21]:
user1 = mu[1]

user3 = mu[3]
user6 = mu[6]
user8 = mu[8]
user10 = mu[10]

In [22]:
user1_watched = user1.dropna()

user3_watched = user3.dropna()
user6_watched = user6.dropna()
user8_watched = user8.dropna()
user10_watched = user10.dropna()

In [23]:
x1_mean = user1_watched.mean()

x3_mean = user3_watched.mean()
x6_mean = user6_watched.mean()
x8_mean = user8_watched.mean()
x10_mean = user10_watched.mean()

In [24]:
mu = user_movie_table.T
mu.head()

userId          1    2    3    4    5    6    7    8    9    10  ...   693  \
       movieId                                                   ...         
rating 1        NaN  NaN  4.0  NaN  NaN  5.0  NaN  4.0  NaN  4.0 ...   NaN   
       2        3.5  NaN  NaN  NaN  3.0  NaN  NaN  NaN  NaN  NaN ...   NaN   
       3        NaN  4.0  NaN  NaN  NaN  3.0  3.0  5.0  NaN  NaN ...   NaN   
       4        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN   
       5        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN   

userId          694  695  696  697  698  699  700  701  702  
       movieId                                               
rating 1        4.5  NaN  NaN  NaN  NaN  4.0  4.0  NaN  3.5  
       2        1.5  NaN  NaN  NaN  5.0  NaN  NaN  NaN  NaN  
       3        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
       4        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
       5        2.5  NaN  NaN  3.5  NaN  2.0  NaN  NaN  NaN  

[5 rows x 702 columns]

In [25]:
mu[3][0] 
#first index is movie3, and second index is index for rows

4.0

In [26]:
numberator = w_1_3*(mu[3][0] - x3_mean) + w_1_6*(mu[6][0] - x6_mean) + w_1_8*(mu[8][0] - x8_mean) + w_1_10*(mu[10][0] - x10_mean)

In [27]:
denominator = w_1_3 + w_1_6 + w_1_8 + w_1_10

In [28]:
#this score will show my prediction for user1 to movie3.
s_1_3 = x1_mean + numberator/denominator

In [29]:
s_1_3  


3.940392569272339